In [1]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langgraph.graph import (
    StateGraph,
    START,
    END
)
from typing_extensions import TypedDict
from IPython.display import Image, display
from pydantic import BaseModel, Field
from typing import List 
import random
import json 
import os 
from template3x4 import (
    get_temp0,
    get_temp1,
    get_temp2,
    get_temp3
)

In [ ]:
class State(TypedDict):
    initial_prompt: str
    category: str 
    story: str
    clue: str 
    solution: str 
    final_prompt: str 

llm = ChatOpenAI(
    api_key="",
    model="gpt-4o-mini"
)

In [ ]:
def extract_json(msg: str):
    start_indx, end_indx = None, None 
    for idx, ch in enumerate(msg):
        if ch == '{':
            start_indx = idx 
            break
    for idx, ch in enumerate(msg):
        if ch == '}':
            end_indx = idx 
    if start_indx == None or end_indx == None:
        return "{}"
    return msg[start_indx: end_indx+1]

In [ ]:
def category_prompt(state: State):
    x, y = state['initial_prompt']['row'], state['initial_prompt']['column']
    msg = llm.invoke(f"""
    You are an AI assistant who is an expert at designing grid logic puzzles. 
    
    I want to create a logic grid puzzle of size {x}*{y}. 
    This means the puzzle should have {x} categories, and each category should contain {y} distinct values.
    One of the category could be a numerical category like money, price, depth or date time. 
    Make the first category in the list of categories as numerical. 
    Rules:
    1. Dont pick weird values for categories. For example orange fruit and orange color, in such cases clues for grid puzzles will become ambiguous.
    2. Some examples of category triplets.
    (Depth of pool, Diver Name, Competition Year),
    (Dog graduation year, Police officer, Dog name),
    (Price, Person, Pet),
    (Price, Person, Fruits),
    ('Distance', 'Exo-Planet', 'Star')
    etc.
    
    Your task:
    1. Generate {x} category names that are intuitive and distinct.
    2. For each category, provide {y} values that are realistic, non-overlapping, and suitable for a logic grid puzzle.
    3. Return the result strictly in **valid JSON** format, with the structure:
    
    {{
        "categories": ["c1", "c2", ..., "c{x}"],
        "c1": ["c1_1", "c1_2", ..., "c1_{y}"],
        "c2": ["c2_1", "c2_2", ..., "c2_{y}"],
        ...
    }}
    No extra commentary or explanation. Only output valid JSON.
    """)
    return {'category': extract_json(msg.content)}

In [ ]:
class Story(BaseModel):
    story: str = Field(
        description = "The story surrounding the logic grid puzzle."
    )
    clue: List[str] = Field(
        description = "The clues to solve the grid puzzle, each entry in a list is an independent clue."
    )

def story_clue_prompt(state: State):
    '''(3, 4) grid puzzles''' 
    def beautify(grid):
        result = ""
        for row in grid:
            temp_row = row.copy()
            temp_row = ' | '.join(row) 
            result += temp_row + "\n"
        return result
    
    elements, dim = 3, 4
    st = json.loads(state['category'])
    keys = st['categories']
    cat = []
    for key in keys:
        row = st[key].copy()
        random.shuffle(row)
        cat.append(row)
    solution = [['' for _ in range(elements)] for __ in range(dim)]
    for i in range(elements):
        for j in range(dim):
            solution[j][i] = cat[i][j]
    soi = list(range(dim))
    random.shuffle(soi)
    problem_templates = [
        get_temp0(),
        get_temp1(),
        get_temp2(),
        get_temp3()
    ]
    index = random.randint(0, len(problem_templates)-1)
    llm.with_structured_output(Story)
    if index == 0:
        fin_prompt = problem_templates[index].format(keys, keys[0], st[keys[0]], \
                keys[1], st[keys[1]], \
                keys[2], st[keys[2]], \
                solution[soi[0]][0], solution[soi[0]][1], solution[soi[0]][2], \
                solution[soi[1]][0], solution[soi[1]][1], solution[soi[1]][2], \
                solution[soi[2]][0], solution[soi[2]][1], solution[soi[2]][2]
            )
    elif index == 1:
        if random.randint(0,2) % 2 == 0:
            var1, var2 = solution[soi[3]][1], solution[soi[2]][2] # (WRONG, RIGHT)
            var3, var4 = solution[soi[3]][0], var1 # (UNK, KNOWN)
        else:
            var1, var2 = solution[soi[2]][1], solution[soi[3]][2] # (RIGHT, WRONG)
            var3, var4 = solution[soi[3]][0], var2 # (UNK, KNOWN)
        fin_prompt = problem_templates[index].format(keys, keys[0], st[keys[0]], \
                keys[1], st[keys[1]], \
                keys[2], st[keys[2]], \
                solution[soi[0]][0], solution[soi[0]][1], solution[soi[0]][2], \
                solution[soi[1]][0], solution[soi[1]][1], solution[soi[1]][2], \
                solution[soi[2]][0], var1, var2, \
                var3, var4
        )
    elif index == 2:
        if random.randint(0, 2) % 2 == 0:
            var1, var2 = solution[soi[3]][1], solution[soi[1]][2] # (WRONG, RIGHT)
            var3, var4 = solution[soi[3]][0], var1 # (UNK, KNOWN)
            var5, var6 = solution[soi[2]][1], solution[soi[3]][2] # (RIGHT, WRONG)
            var7, var8 = solution[soi[3]][1], var6 # (UNK, KNOWN)
        else:
            var1, var2 = solution[soi[1]][1], solution[soi[3]][2] # (RIGHT, WRONG)
            var3, var4 = solution[soi[3]][1], var2 # (UNK, KNOWN)
            var5, var6 = solution[soi[3]][1], solution[soi[2]][2] # (WRONG, RIGHT)
            var7, var8 = solution[soi[3]][0], var5 # (UNK, KNOWN)
        fin_prompt = problem_templates[index].format(keys, keys[0], st[keys[0]], \
                keys[1], st[keys[1]], \
                keys[2], st[keys[2]], \
                solution[soi[0]][0], solution[soi[0]][1], solution[soi[0]][2], \
                solution[soi[1]][0], var1, var2, \
                solution[soi[2]][0], var5, var6, \
                var3, var4, \
                var7, var8
        )
    elif index == 3:
        r_cat = keys[1]
        fin_prompt = problem_templates[index].format(keys, keys[0], st[keys[0]], \
                keys[1], st[keys[1]], \
                keys[2], st[keys[2]], \
                r_cat, solution[1][2], solution[0][1], \
                solution[1][1], r_cat, solution[0][2], \
                solution[2][1], solution[2][0], solution[0][2], \
                solution[1][0], solution[1][2], \
                solution[3][0], solution[3][2]
        )
    resp = llm.invoke(fin_prompt)
    resp = json.loads(extract_json(resp.content))
    print(f'INDEX[{index}]; prompt> {fin_prompt}]')
    return {
        "solution": beautify(solution), 
        "story": resp['story'],
        "clue": resp['clues']
    }

'''TEST CODE'''
# state = {
#     'category': json.dumps({
#         "categories": ["Month", "Dog", "Officer"],
#         "Month": ["March", "April", "May", "June"],
#         "Dog": ["Tinkerbell", "Barca", "McGruff", "Sarge"],
#         "Officer": ["Trujillo", "Salinas", "Zimmerman", "Nolan"]
#     }, indent=4)
# }
# story_clue_prompt(state)

'TEST CODE'

In [5]:

def final_prompt(state: State):
    st = json.loads(state['category'])
    keys = st['categories']
    clues = state['clue']
    if isinstance(clues, list):
        clues = '\n'.join(clues)
    post_fix = f'''{state['story']}
Categories:
{keys}

{keys[0]}: {st[keys[0]]}
{keys[1]}: {st[keys[1]]}
{keys[2]}: {st[keys[2]]}

Clues:
{clues}


While answering use the following format:
Step-by-step solution:
Your steps showing how you are solving the puzzle
Final Answer:
Create a table like this
c1_1 | c2_1 | c3_1
c1_2 | c2_2 | c3_2
c1_3 | c2_3 | c3_3
c1_4 | c2_4 | c3_4'''
    return {
        "final_prompt": post_fix
        }

In [6]:
workflow = StateGraph(State)
workflow.add_node('c_prompt', category_prompt)
workflow.add_node('sc_prompt', story_clue_prompt)
workflow.add_node('f_prompt', final_prompt)

workflow.add_edge(START, 'c_prompt')
workflow.add_edge('c_prompt', 'sc_prompt')
workflow.add_edge('sc_prompt', 'f_prompt')
workflow.add_edge('f_prompt', END)

chain = workflow.compile()

In [11]:
state = chain.invoke({"initial_prompt":{
        "row": 3,
        "column": 4
        }
})

INDEX[3]; prompt> Form a grid puzzle using the following template. 
A logic grid puzzle should have a story and clues.

Categories:
['Price', 'Person', 'Pet']

Price: ['$150', '$200', '$250', '$300']
Person: ['Alice', 'Bob', 'Carol', 'David']
Pet: ['Cat', 'Dog', 'Bird', 'Fish']

Clues:
Person of Bird category relation with Bob
Carol category relation with Person of Dog
David is either $250 or Dog
$200 and Bird are related
$300 and Cat are related 

Your job is to fill in the following values in the following JSON.
{
    "story": "",
    "clues": "",
}
No extra commentary or explanation. Only output valid JSON.]


In [12]:
print(state['final_prompt'], state['solution'], \
        sep="\n-------------\n")

Four friends - Alice, Bob, Carol, and David - each bought a different pet (Cat, Dog, Bird, or Fish) at different prices ($150, $200, $250, or $300). Using the clues below, determine who bought which pet and at what price.
Categories:
['Price', 'Person', 'Pet']

Price: ['$150', '$200', '$250', '$300']
Person: ['Alice', 'Bob', 'Carol', 'David']
Pet: ['Cat', 'Dog', 'Bird', 'Fish']

Clues:
1. The person who bought the Bird is not Bob. 2. Carol did not buy the Dog. 3. David bought either the $250 pet or the Dog. 4. The $200 pet and the Bird were bought by the same person. 5. The $300 pet and the Cat were bought by the same person.


While answering use the following format:
Step-by-step solution:
Your steps showing how you are solving the puzzle
Final Answer:
Create a table like this
c1_1 | c2_1 | c3_1
c1_2 | c2_2 | c3_2
c1_3 | c2_3 | c3_3
c1_4 | c2_4 | c3_4
-------------
$150 | Bob | Dog
$200 | Carol | Bird
$250 | David | Fish
$300 | Alice | Cat



In [13]:
import pandas as pd 

df = pd.read_csv('data/grid_puzzle_easy_x.csv')
new_row = pd.DataFrame({'question': [state['final_prompt']], 'answer': [state['solution']]})
df = pd.concat([df, new_row], ignore_index=True)
df.to_csv('data/grid_puzzle_easy_x.csv', index=False)

In [14]:
df.tail()

,question,answer
11,"Kara recently moved to a new city, so last mon...",22 | Alton | banker\n23 | Brent | accountant\n...
12,"Four friends - Alice, Bob, Carol, and David - ...",$250 | Alice | Bird\n$200 | David | Fish\n$150...
13,"Four friends - Alice, Bob, Carol, and David - ...",$300 | Bob | Dog\n$200 | Alice | Cat\n$250 | C...
14,"Four friends - Alice, Bob, Carol, and David - ...",$200 | Carol | Cat\n$300 | Bob | Fish\n$250 | ...
15,"Four friends - Alice, Bob, Carol, and David - ...",$150 | Bob | Dog\n$200 | Carol | Bird\n$250 | ...
